# Background

This notebook is a testbed for building a RAG for AdvisorAlly

# Import Libraries and Dependencies

In [1]:
import os
os.chdir("../")

In [2]:
import os
from typing import List
import sqlite3
import boto3

import streamlit as st
import streamlit_authenticator as stauth
import yaml
from bs4 import BeautifulSoup as Soup
import pandas as pd

from dotenv import load_dotenv

# load agents and tools modules
from langchain.agents import AgentType, Tool, create_sql_agent, initialize_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.schema.runnable import RunnableConfig
from langchain.callbacks import StreamlitCallbackHandler
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory
from langchain.sql_database import SQLDatabase
from langchain.text_splitter import CharacterTextSplitter
from langchain.tools import DuckDuckGoSearchResults, Tool
from langchain.vectorstores import FAISS
from loguru import logger
from yaml.loader import SafeLoader

# load_dotenv("secrets.env")
load_dotenv(".streamlit/secrets.toml")

True

# Index PDF into vectorstore

In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
loader = PyPDFDirectoryLoader("./data/pdfs/")
pages = loader.load()


In [4]:
text_splitter = CharacterTextSplitter(chunk_size=1000, separator="\n")
docs, metadatas = [], []
for page in pages:
    splits = text_splitter.split_text(page.page_content)
    docs.extend(splits)
    metadatas.extend([{"source": page.metadata["source"], "page": page.metadata["page"]}] * len(splits))
    logger.info(f"Split {page.metadata['source']} page {page.metadata['page']} into {len(splits)} chunks")

2023-10-29 12:30:19.103 | INFO     | __main__:<module>:7 - Split data/pdfs/HSBCLife-happymummy-happyfamily-brochure.pdf page 0 into 1 chunks
2023-10-29 12:30:19.104 | INFO     | __main__:<module>:7 - Split data/pdfs/HSBCLife-happymummy-happyfamily-brochure.pdf page 1 into 4 chunks
2023-10-29 12:30:19.105 | INFO     | __main__:<module>:7 - Split data/pdfs/HSBCLife-happymummy-happyfamily-brochure.pdf page 2 into 5 chunks
Created a chunk of size 1125, which is longer than the specified 1000
2023-10-29 12:30:19.105 | INFO     | __main__:<module>:7 - Split data/pdfs/HSBCLife-happymummy-happyfamily-brochure.pdf page 3 into 10 chunks
2023-10-29 12:30:19.106 | INFO     | __main__:<module>:7 - Split data/pdfs/HSBCLife-happymummy-happyfamily-brochure.pdf page 4 into 5 chunks
2023-10-29 12:30:19.107 | INFO     | __main__:<module>:7 - Split data/pdfs/hsbc-life-treasure-brochure-english.pdf page 0 into 1 chunks
2023-10-29 12:30:19.107 | INFO     | __main__:<module>:7 - Split data/pdfs/hsbc-life-tre

In [5]:
embeddings_path = "./data/embeddings/"
embeddings_file_name = "pdfs_vectorstore.pkl"

if not os.path.exists(os.path.join(embeddings_path, embeddings_file_name)):
    store = FAISS.from_texts(docs, OpenAIEmbeddings(), metadatas=metadatas)
    store.save_local(os.path.join(embeddings_path, embeddings_file_name))
else:
    store = FAISS.load_local(os.path.join(embeddings_path, embeddings_file_name), OpenAIEmbeddings())

llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4", temperature=0.0)

# initialize vectorstore retriever object
pdf_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=store.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 6}),
)

In [6]:
pdf_chain.run("Tell me about HSBC Life Insurance products")

'The context provided mentions a product from HSBC Life called Life Treasure (II). This is a comprehensive protection plan designed to cover your whole life. For more detailed information about this product or other insurance products from HSBC Life, you can visit their website at www.hsbclife.com.sg or call their enquiry line at (65) 6880 4888.'

# index data into sqlite3 database

This has been made into a one-time runnable script `src/create_sqlite_db.py`

In [ ]:
# write pandas code to read xlsx file
import pandas as pd
df = pd.read_excel('data/scrapped_output_term_full_20231011.xlsx', sheet_name='Term Life', index_col=0)
# df = df.sample(frac=0.005, replace=False, random_state=1)

In [13]:
# Create a connection to the SQLite database (it will create a new file if it doesn't exist)
conn = sqlite3.connect("data/sqlite/my_database.sqlite")

# Write the DataFrame to the SQLite database
df.to_sql("term_life_table", conn, if_exists="replace", index=True)

# Close the connection
conn.close()

In [16]:
# Connect to the SQLite database
conn = sqlite3.connect("data/sqlite/my_database.sqlite")
cursor = conn.cursor()

# Run a SQL query
cursor.execute("SELECT * FROM term_life_table WHERE AGE = 40")
results = cursor.fetchall()

# Do something with the results...
# print(results)

# Close the connection
conn.close()

# Initialize SQL Agent

In [16]:
db = SQLDatabase.from_uri("sqlite:///data/sqlite/my_database.sqlite")
toolkit = SQLDatabaseToolkit(
    db=db,
    llm=ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4", temperature=0.0),
)

sql_agent = create_sql_agent(
    llm=ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4", temperature=0.0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

# Index webpages into vectorstore

In [10]:
import xmltodict
import requests
from bs4 import BeautifulSoup

def extract_text_from(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, features="html.parser")
    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    return '\n'.join(line for line in lines if line)

pages = []

url = "https://www.moneysmart.sg/cancer-insurance"
pages.append({'text': extract_text_from(url), 'source': url})

In [11]:
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup

url = "https://www.moneysmart.sg/cancer-insurance"
loader = RecursiveUrlLoader(url=url, max_depth=3, extractor=lambda x: Soup(x, "html.parser").text)
pages = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1500, separator="\n")
docs, metadatas = [], []
for page in pages:
    splits = text_splitter.split_text(page.page_content)
    docs.extend(splits)
    metadatas.extend([{"source": page.metadata["source"]}] * len(splits))
    logger.info(f"Split {page.metadata['source']} into {len(splits)} chunks")

In [13]:
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embeddings_path = "./data/embeddings/"
embeddings_file_name = "webpages_vectorstore.pkl"

if not os.path.exists(os.path.join(embeddings_path, embeddings_file_name)):
    store = FAISS.from_texts(docs, OpenAIEmbeddings(), metadatas=metadatas)
    store.save_local(os.path.join(embeddings_path, embeddings_file_name))
else:
    store = FAISS.load_local(os.path.join(embeddings_path, embeddings_file_name), OpenAIEmbeddings())

llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4", temperature=0.0)

# initialize vectorstore retriever object
webpage_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=store.as_retriever(),
)

In [14]:
webpage_chain.run("Tell me about MSIG cancer insurance")


'MSIG cancer insurance is a policy that covers various stages of cancer along with medical expenses. It can be purchased directly online with a hassle-free approach and at an affordable price. The maximum sum insured for a non-smoker male of 25 years is S$100,000. The insurance plan, called MSIG CancerCare Plus, offers coverage from early to late stages of cancer. The annual premium for this plan is S$54, but there is currently a 20% discount, making the premium S$43.20. There is also a promotion where you can get S$10 cash via PayNow as fast as 1 month while enjoying the 20% off on MSIG CancerCare Plus Insurance plans. This promotion is valid until 31 Oct 2023. Please note that this plan does not include a death benefit.'

# DuckDuckGo Agent

In [8]:
from langchain.tools import Tool, DuckDuckGoSearchResults
from langchain.utilities import DuckDuckGoSearchAPIWrapper

# wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", time="d", max_results=5)
# ddg_search = DuckDuckGoSearchResults()
ddg_agent = DuckDuckGoSearchResults(
    name="Web Search Results",
    backend="api",
    description="""A wrapper around Duck Duck Go Search. Useful for when you need to answer when there are no more alternatives. 
    Input should be a search query. Output is a JSON array of the query results. 
    Please search in Singapore region for best results.
    """,
)

# Initialize LLM agent

2 kinds:
- AgentType.ZERO_SHOT_REACT_DESCRIPTION
- AgentType.CONVERSATIONAL_REACT_DESCRIPTION

app.py is using `AgentType.CONVERSATIONAL_REACT_DESCRIPTION`

In [17]:
tools = [
    Tool(
        name="Insurance Policies Brochure",
        func=pdf_chain.run,
        description="""
        Useful for when you need to answer questions about insurance products from HSBC. 
        Do not say that it's recommended to seek advise from a financial planner or advisor before making a decision, because the chatbot user is a financial planner. 
        """,
    ),
    Tool(
        name="Term Life Polices",
        func=sql_agent.run,
        description=f"""
        Useful for when you need to answer questions about premium for term life products data stored in sql table term_life_table.
        Table name has no quotes. 
        Run sql operations on table to help you get the right answer.
        """,
    ),
    Tool(
        name="Cancer Insurance Plans 2023",
        func=webpage_chain.run,
        description=f"""
        Useful for when you need to answer questions about cancer insurance plans in singapore too.  
        """,
    ),
    ddg_agent,
]

---

In [18]:
# change the value of the prefix argument in the initialize_agent function. This will overwrite the default prompt template of the zero shot agent type
agent_kwargs = {'prefix': f'You are friendly financial advisor that advises on insurance polices. You are tasked to assist the current user on questions related to term life policies. You have access to the following tools:'}


# initialize the LLM agent
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True, 
                         max_iterations=10,
                         early_stopping_method="generate",
                         agent_kwargs=agent_kwargs
                         )
# define q and a function for frontend
def get_response(user_input):
    response = agent.run(user_input)
    return response

In [19]:
print(agent.agent.llm_chain.prompt.template)

You are friendly financial advisor that advises on insurance polices. You are tasked to assist the current user on questions related to term life policies. You have access to the following tools:

Insurance Policies Brochure: 
        Useful for when you need to answer questions about insurance products from HSBC. 
        Do not say that it's recommended to seek advise from a financial planner or advisor before making a decision, because the chatbot user is a financial planner. 
        
Term Life Polices: 
        Useful for when you need to answer questions about premium for term life products data stored in sql table term_life_table.
        Table name has no quotes. 
        Run sql operations on table to help you get the right answer.
        
Cancer Insurance Plans 2023: 
        Useful for when you need to answer questions about cancer insurance plans in singapore too.  
        
Web Search Results: A wrapper around Duck Duck Go Search. Useful for when you need to answer when t

# Test Queries

In [20]:
get_response("what is a term life product?")



> Entering new AgentExecutor chain...
The question is about the definition of a term life product. I can find this information in the Insurance Policies Brochure.
Action: Insurance Policies Brochure
Action Input: term life product
Observation: The context provided does not include information about a term life product. It only discusses a whole life insurance plan, specifically the HSBC Life - Life Treasure (II).
Thought:The Insurance Policies Brochure does not provide information on term life products. I should use the Web Search Results to find a definition.
Action: Web Search Results
Action Input: what is a term life product
Observation: [snippet: Term life insurance guarantees payment of a stated death benefit to the insured's beneficiaries if the insured person dies during the specified term. These policies have no value other than the..., title: Term Life Insurance: What It Is, Different Types, Pros and Cons, link: https://www.investopedia.com/terms/t/termlife.asp], [snippet: T

'Term life insurance is a type of life insurance policy that provides coverage for a certain period of time, or a specified "term" of years. If the insured dies during this time period, the insurance company guarantees payment of a stated death benefit to the insured\'s beneficiaries. These policies have no value other than the guaranteed death benefit.'

In [21]:
get_response("which term life product has most expensive annual premium for 40 year old male?")



> Entering new AgentExecutor chain...
I need to check the term life policies data to find out which product has the most expensive annual premium for a 40 year old male.
Action: Term Life Polices
Action Input: SELECT product_name, MAX(annual_premium) FROM term_life_table WHERE age = 40 AND gender = 'male'

> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ''
Observation: term_life_table
Thought:The term_life_table seems to be the relevant table for the query. I should check its schema to understand its structure and the fields it contains.
Action: sql_db_schema
Action Input: 'term_life_table'
Observation: Error: table_names {"'term_life_table'"} not found in database
Thought:It seems like there was a mistake in the table name. I should try again without the quotes.
Action: sql_db_schema
Action Input: term_life_table
Observation: 
CREATE TABLE term_life_table (
	"index" INTEGER, 
	"PRODUCT_CATE" TEXT, 
	"CATEGORY" TEXT, 
	"AGE" INTEGER, 
	"GENDER" TEXT, 
	

"The term life product with the most expensive annual premium for a 40 year old male is 'PRUActive Term' with an annual premium of 72002."

In [ ]:
get_response("give me the range for annual premium for term life products for a 35 year old non-smoker female?")

In [22]:

# get_response("tell me what you know about cancer insurance plan in singapore 2023")
get_response("Tell me about MSIG cancer insurance")



> Entering new AgentExecutor chain...
The user is asking about a specific cancer insurance plan from MSIG. I should use the Cancer Insurance Plans 2023 tool to find information about this plan.
Action: Cancer Insurance Plans 2023
Action Input: MSIG cancer insurance
Observation: MSIG offers a cancer insurance plan called CancerCare Plus. This plan covers stages of cancer along with medical expenses. The maximum sum insured is S$100,000. The annual premium for a non-smoker male of 25 years is S$43.20 after a 20% discount. The coverage includes early to late stage cancer. There is no death benefit mentioned. You can purchase this insurance directly online.
Thought:I now have the information about the MSIG cancer insurance plan.
Final Answer: MSIG offers a cancer insurance plan called CancerCare Plus. This plan covers stages of cancer along with medical expenses. The maximum sum insured is S$100,000. The annual premium for a non-smoker male of 25 years is S$43.20 after a 20% discount. Th

'MSIG offers a cancer insurance plan called CancerCare Plus. This plan covers stages of cancer along with medical expenses. The maximum sum insured is S$100,000. The annual premium for a non-smoker male of 25 years is S$43.20 after a 20% discount. The coverage includes early to late stage cancer. There is no death benefit mentioned. You can purchase this insurance directly online.'

# Debug

In [34]:
from langchain import hub
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
# prompt = hub.pull("hwchase17/react-chat-json")

In [38]:
# prompt = prompt.partial(
#     tools=render_text_description(tools),
#     tool_names=", ".join([t.name for t in tools]),
#     template="""
# You are friendly financial advisor that advises on insurance polices. You are tasked to assist the current user on questions related to term life policies. You have access to the following tools:

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [MAS term life Policies, Term Life Polices, Cancer Insurance Plans 2023, DuckDuckGo Results JSON]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin!

# Question: {input}
# Thought:{agent_scratchpad}

# """,
# )
# llm_with_stop = llm.bind(stop=["\nObservation"])
# agent = (
#     {
#         "input": lambda x: x["input"],
#         "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
#         "chat_history": lambda x: x["chat_history"],
#     }
#     | prompt
#     | llm_with_stop
#     | ReActSingleInputOutputParser()
# )

memory = ConversationBufferMemory(memory_key="chat_history")
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory

# change the value of the prefix argument in the initialize_agent function. This will overwrite the default prompt template of the zero shot agent type
agent_kwargs = {
    "prefix": f"You are friendly financial advisor that advises on insurance polices. You are tasked to assist the current user on questions related to term life policies. You have access to the following tools:"
}

agent_executor = initialize_agent(
    tools,
    llm=ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4", temperature=0.0),
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    max_iterations=20,
    early_stopping_method="generate",
    handle_parsing_errors=True,
    agent_kwargs=agent_kwargs,
)


def get_response(user_input):
    MAX_RETRIES = 5  # Set the maximum number of retries

    for attempt in range(MAX_RETRIES):
        try:
            # agent_response = agent_chain.run(input)
            agent_response = agent_executor.invoke({"input": user_input})["output"]

        except ValueError as e:
            response = str(e)

            if not response.startswith("Could not parse LLM output: `"):
                raise e

            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

            input = response  # Feed the response back into the agent

        # If we've reached the maximum number of retries, give up
        if attempt == MAX_RETRIES - 1:
            return "Error: Agent failed to run after multiple attempts"
        return agent_response
    # try:
    #     response = agent_executor.invoke({"input": user_input})["output"]
    # except ValueError as e:
    #     response = str(e)
    #     if not response.startswith("Could not parse LLM output: `"):
    #         raise e
    #     response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")
    # return response

In [39]:
agent_executor.invoke({"input": "hi, i am bob"})['output']
# get_response("hi, i am bob")
# agent_executor.run("hi, i am bob")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello Bob! How can I assist you with your insurance policy queries today?

> Finished chain.


'Hello Bob! How can I assist you with your insurance policy queries today?'

In [40]:
agent_executor.invoke({"input": "What is my name?"})['output']
# get_response("hi, i am bob")
# agent_executor.run("hi, i am bob")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Your name is Bob.

> Finished chain.


'Your name is Bob.'

In [41]:

# agent_executor.run("what is a direct purchase insurance product?")
agent_executor.invoke({"input": "what is a direct purchase insurance product?"})['output']



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: A Direct Purchase Insurance (DPI) product is a type of life insurance policy that you can buy directly from an insurance company, without the need for a financial advisor or insurance agent. These policies are designed to be simple and easy to understand, and they often come with lower premiums because there are no commission fees. DPI products can include term life insurance, whole life insurance, and other types of coverage.

> Finished chain.


'A Direct Purchase Insurance (DPI) product is a type of life insurance policy that you can buy directly from an insurance company, without the need for a financial advisor or insurance agent. These policies are designed to be simple and easy to understand, and they often come with lower premiums because there are no commission fees. DPI products can include term life insurance, whole life insurance, and other types of coverage.'

In [42]:

# agent_executor.run("what is a direct purchase insurance product?")
agent_executor.invoke({"input": "give me the range for annual premium for term life products for a 35 year old non-smoker female?"})['output']
# get_response("give me the range for annual premium for term life products for a 35 year old non-smoker female?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Term Life Polices
Action Input: SELECT MIN(annual_premium), MAX(annual_premium) FROM term_life_table WHERE age = 35 AND smoker = 'No' AND gender = 'Female'

> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ''
Observation: term_life_table
Thought:The term_life_table is available. Now I need to check the schema of this table to ensure it contains the columns I need for the query.
Action: sql_db_schema
Action Input: 'term_life_table'
Observation: Error: table_names {"'term_life_table'"} not found in database
Thought:It seems like there was a mistake in the table name. I should try again without the quotes.
Action: sql_db_schema
Action Input: term_life_table
Observation: 
CREATE TABLE term_life_table (
	"index" INTEGER, 
	"PRODUCT_CATE" TEXT, 
	"CATEGORY" TEXT, 
	"AGE" INTEGER, 
	"GENDER" TEXT, 
	"SMOKER" TEXT, 
	"CI_BENEFIT" TEXT, 
	"PREMIUM_TYPE" TEXT, 
	"COVERAGE_TERM_OPTI

'The range for the annual premium for term life products for a 35-year-old non-smoker female is from 0 to 39125.'

In [ ]:
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')

# Define the bucket name and the file key
bucket_name = "for-singlife"
file_key = "projects/advisorally/scrapped_output_term_full_20231011.xlsx"
download_path = "data/excel/scrapped_output_term_full_20231011.xlsx" 

# Download the file
s3.download_file(bucket_name, file_key, download_path)

print(f"Downloaded {file_key} from {bucket_name} to {download_path}")


Downloaded projects/advisorally/scrapped_output_term_full_20231011.xlsx from for-singlife to data/excel/scrapped_output_term_full_20231011.xlsx
